In [32]:
# Necessary imports
import pandas as pd
import numpy as np
import pickle

from tqdm import notebook

# Import path of the Geocoding Validation project
import sys
PATH = r"""C:\Users\andrew.tan\Jupyter Notebooks\Geocoding_Validation\\"""
sys.path.insert(0, PATH)

from gc_val import GC_Val

val = GC_Val(PATH + "all_data.csv", PATH + "neighborhoods.pkl")

false_responses = []
fr = pickle.load(open(PATH + "false_responses_all.pkl", "rb"))
false_responses = [i[0] for i in fr]


In [35]:
# Retrieve the desired features from data
X = pd.DataFrame(columns=['index', 'ratio', 'calculation_method', 'confidence', 'match_codes', 'expected'])

for i in notebook.tqdm(range(len(val.df))):
    # Ratio
    try:
        ratio = val.get_ratio(i)
    except KeyError as e:
        ratio = 0
    
    # Calculation_method
    # ['Rooftop', 'InterpolationOffset', 'Interpolation'] = [0, 1, 2]
    method = val.get_calculation_method(i)
    if method == 'Rooftop': method = 0
    elif method == 'InterpolationOffset': method = 1
    elif method == 'Interpolation': method = 2
    else: method = None
    
    # Confidence
    # ['High', 'Medium', 'Low'] = [0, 1, 2]
    conf = val.get_confidence(i)
    if conf == 'High': conf = 0
    elif conf == 'Medium': conf = 1
    elif conf == 'Low': conf = 2
    else: conf = None
    
    # Match_codes
    # [['UpHierarchy'], ['Ambiguous', 'UpHierarchy'], ['Good'], ['Ambiguous']]= [0, 1, 2, 3]
    code = str(val.get_match_codes(i))
    if code == "['UpHierarchy']": code = 0
    elif code == "['Ambiguous', 'UpHierarchy']": code = 1
    elif code == "['Good']": code = 2
    elif code == "['Ambiguous']": code = 3
    else: code = None
    
    expected = -1
    if i in false_responses:  # 0 for valid and 1 for false entries
        expected = 1
    else:
        expected = 0
        
    insert = [i, ratio, method, conf, code, expected]
    X.loc[i] = insert
    
    
    

  0%|          | 0/199986 [00:00<?, ?it/s]

In [44]:
X

,index,ratio,calculation_method,confidence,match_codes,expected
0,0.0,1.000000,0.0,0.0,2.0,0.0
1,1.0,0.960000,0.0,0.0,2.0,0.0
2,2.0,0.933333,0.0,0.0,2.0,0.0
3,3.0,0.956522,0.0,0.0,2.0,0.0
4,4.0,1.000000,0.0,0.0,2.0,0.0
...,...,...,...,...,...,...
199981,199981.0,1.000000,0.0,0.0,2.0,0.0
199982,199982.0,0.869565,0.0,0.0,2.0,0.0
199983,199983.0,1.000000,0.0,0.0,2.0,0.0
199984,199984.0,1.000000,0.0,0.0,2.0,0.0


In [17]:
import traceback
distances = pickle.load(open("new_distances.pkl", "rb"))
try:
    for i in notebook.tqdm(range(len(distances), len(val.df))):
        given, resp = val.get_addresses(i)
        dist = val.get_distance(given, resp)
        distances.append(dist)
except Exception as e:
    traceback.print_exc()
finally:
    pickle.dump(distances, open("new_distances.pkl", "wb"))

  0%|          | 0/97851 [00:00<?, ?it/s]

Index error
Prompt: http://dev.virtualearth.net/REST/v1/Locations?countryRegion=US&adminDistrict2=dukes county&postalCode=02568&locality=oak bluffs&adminDistrict=ma&key=ApSkqZuWjYH5CoKaFuXUQ2Kn7B6q0waAAQHR7mHgLRsCl324fhyM4pO_k_rCxyNv


In [45]:
distances1 = pickle.load(open("distances.pkl", "rb"))
distances2 = pickle.load(open("distances2.pkl", "rb"))
for d in distances2:
    distances1.append(d)
pickle.dump(distances1, open("all_distances.pkl", "wb"))

In [46]:
X['distances'] = distances1
X

,index,ratio,calculation_method,confidence,match_codes,expected,distances
0,0.0,1.000000,0.0,0.0,2.0,0.0,0.0
1,1.0,0.960000,0.0,0.0,2.0,0.0,0.0
2,2.0,0.933333,0.0,0.0,2.0,0.0,0.0
3,3.0,0.956522,0.0,0.0,2.0,0.0,0.0
4,4.0,1.000000,0.0,0.0,2.0,0.0,0.0
...,...,...,...,...,...,...,...
199981,199981.0,1.000000,0.0,0.0,2.0,0.0,0.0
199982,199982.0,0.869565,0.0,0.0,2.0,0.0,0.0
199983,199983.0,1.000000,0.0,0.0,2.0,0.0,0.0
199984,199984.0,1.000000,0.0,0.0,2.0,0.0,0.0


In [47]:
# Save the combined data set
pickle.dump(X, open("all_data.pkl", "wb"))